In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 204 plant ae data

### Reading the files and extracting relevant information:

In [5]:
plant = "AQ"
xls_files = {}


xls = pd.ExcelFile(
    f"../../../../data/raw/204/EMBRAPII hubIC IACC CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  AQ ['INSTRUÇÕES', 'Fibro', '7', '153', '89']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[1:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])
        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('AQ', 'Fibro', array(['', 'Fibro'], dtype=object)),
 ('AQ', '7', array(['', 'CPIIF40'], dtype=object)),
 ('AQ', '153', array(['', 'CPIIF32'], dtype=object)),
 ('AQ', '89', array(['', 'CPVARI'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'': 4})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[1:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [13]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [14]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

In [15]:
df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

<h3>Percentage of missing values:</h3>
    
    
<table>
    <th>Plant AQ - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Specific Gravity</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Insoluble Residue</td>
        <td>100.00</td>
    </tr>     
    <tr>
        <td>TiO2</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>#400</td>
        <td>100.00</td>
    </tr>
</table>

In [18]:
FEATRUES_TO_DROP = ["Specific Gravity", "Insoluble Residue", "TiO2", "#400"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [19]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [20]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [21]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [22]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

In [23]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
CS1,50.232558
Na2O,43.372093
MgO,1.046512
SiO2,1.046512
SO3,1.046512
K2O,1.046512
Fe2O3,1.046512
CaO,1.046512
Al2O3,0.930233
Loss on Ignition,0.930233


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>

Upon removing samples with zero value and nans on the target variable, the dataset becomes smaller and hence the missing of other variables become large. Hence, here we remove more variables.
    
<table>
    <th>Plant AQ - Full Dataset:</th>
    <th></th>
    <tr>
        <td>CS1</td>
        <td>50.23</td>
    </tr>
    <tr>
        <td>Na2O</td>
        <td>43.27</td>
    </tr>     
</table>

In [24]:
FEATRUES_TO_DROP_2 = ["CS1", "Na2O"]
df = df.drop(labels=FEATRUES_TO_DROP_2, axis=1)

<IPython.core.display.Javascript object>

In [25]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
MgO,1.046512
SiO2,1.046512
SO3,1.046512
K2O,1.046512
Fe2O3,1.046512
CaO,1.046512
Al2O3,0.930233
Loss on Ignition,0.930233
Blaine,0.232558
Final setting time,0.232558


<IPython.core.display.Javascript object>

In [26]:
df = df.dropna()

<IPython.core.display.Javascript object>

In [27]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Date,0.000000
CaO,0.000000
CS28,0.000000
CS7,0.000000
CS3,0.000000
Final setting time,0.000000
Initial setting time,0.000000
#325,0.000000
#200,0.000000
Blaine,0.000000


<IPython.core.display.Javascript object>

### Dropping duplicated rows

In [28]:
df.shape

(848, 18)

<IPython.core.display.Javascript object>

In [29]:
df = df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [30]:
df = df.loc[df.select_dtypes("number").drop_duplicates().index].reset_index(drop=True)

chemical_vars = [
    "CaO",
    "MgO",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "Fe2O3",
    "Loss on Ignition",
]

df = df.loc[
    df.select_dtypes("number").drop_duplicates(subset=chemical_vars).index
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [31]:
df.shape

(818, 18)

<IPython.core.display.Javascript object>

### Dropping rows with negative values

In [32]:
df[df.drop(["Date"], axis=1).duplicated()].drop("Date", axis=1).select_dtypes(
    include="number"
).apply(lambda x: x < 0).sum().sort_values(ascending=False)

CaO                     0.0
MgO                     0.0
Al2O3                   0.0
SiO2                    0.0
SO3                     0.0
K2O                     0.0
Fe2O3                   0.0
Loss on Ignition        0.0
Blaine                  0.0
#200                    0.0
#325                    0.0
Initial setting time    0.0
Final setting time      0.0
CS3                     0.0
CS7                     0.0
CS28                    0.0
dtype: float64

<IPython.core.display.Javascript object>

### Removing outliers

In [33]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,818,2021-05-11 16:18:46.650366720,2020-01-03 00:00:00,2021-03-03 00:00:00,2023-06-12 19:55:12,2023-07-03 00:00:00,NaN
CaO,818.0,60.714401,32.14,60.94,63.0283,66.67,1.695482
MgO,818.0,1.04533,0.29,1.04,1.4815,4.22,0.168303
Al2O3,818.0,3.951858,2.23,3.98,4.4383,4.69,0.25284
SiO2,818.0,18.580685,1.06,18.63,20.2549,21.35,0.936644
SO3,818.0,2.953594,1.92,2.86,4.2483,4.72,0.424814
K2O,818.0,0.330929,0.1,0.32,0.4583,6.31,0.227179
Fe2O3,818.0,2.469535,1.03,2.44,2.9349,19.08,0.619109
Loss on Ignition,818.0,7.827176,3.32,8.23,12.1349,12.5,2.621185
Blaine,818.0,4211.735941,3590.0,4150.0,5058.3,5650.0,301.330658


<IPython.core.display.Javascript object>

In [34]:
df = df.reset_index(drop=True)
df = df[df["MgO"] < 2].reset_index(drop=True)
df = df[df["K2O"] < 0.5].reset_index(drop=True)
df = df[df["Fe2O3"] < 3].reset_index(drop=True)
df = df[df["#200"] < 4].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [35]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,801,2021-05-04 22:31:54.606741504,2020-01-03 00:00:00,2021-02-26 00:00:00,2023-06-13 00:00:00,2023-07-03 00:00:00,NaN
CaO,801.0,60.728752,32.14,60.98,63.02,66.67,1.696746
MgO,801.0,1.036966,0.72,1.04,1.32,1.68,0.096154
Al2O3,801.0,3.952197,2.23,3.98,4.44,4.69,0.253586
SiO2,801.0,18.599076,16.06,18.63,20.23,21.35,0.707712
SO3,801.0,2.951823,1.92,2.86,4.25,4.72,0.423547
K2O,801.0,0.319039,0.1,0.32,0.41,0.47,0.042956
Fe2O3,801.0,2.445206,1.03,2.43,2.89,2.98,0.207053
Loss on Ignition,801.0,7.804357,3.32,8.18,12.14,12.5,2.630606
Blaine,801.0,4211.310861,3590.0,4150.0,5060.0,5650.0,301.497769


<IPython.core.display.Javascript object>

In [36]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,801,2021-05-04 22:31:54.606741504,2020-01-03 00:00:00,2020-07-20 00:00:00,2021-02-26 00:00:00,2021-12-06 00:00:00,2023-07-03 00:00:00,NaN
CaO,801.0,60.728752,32.14,59.92,60.98,61.75,66.67,1.696746
MgO,801.0,1.036966,0.72,0.98,1.04,1.09,1.68,0.096154
Al2O3,801.0,3.952197,2.23,3.77,3.98,4.14,4.69,0.253586
SiO2,801.0,18.599076,16.06,18.2,18.63,19.05,21.35,0.707712
SO3,801.0,2.951823,1.92,2.65,2.86,3.18,4.72,0.423547
K2O,801.0,0.319039,0.1,0.3,0.32,0.34,0.47,0.042956
Fe2O3,801.0,2.445206,1.03,2.3,2.43,2.6,2.98,0.207053
Loss on Ignition,801.0,7.804357,3.32,5.25,8.18,10.33,12.5,2.630606
Blaine,801.0,4211.310861,3590.0,4000.0,4150.0,4380.0,5650.0,301.497769


<IPython.core.display.Javascript object>

In [37]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,801,NaN,NaN,NaN,2021-05-04 22:31:54.606741504,2020-01-03 00:00:00,2020-07-20 00:00:00,2021-02-26 00:00:00,2021-12-06 00:00:00,2023-07-03 00:00:00
Cement_Type,801,4,CPII F32,411,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  801 non-null    datetime64[ns]
 1   CaO                   801 non-null    float64       
 2   MgO                   801 non-null    float64       
 3   Al2O3                 801 non-null    float64       
 4   SiO2                  801 non-null    float64       
 5   SO3                   801 non-null    float64       
 6   K2O                   801 non-null    float64       
 7   Fe2O3                 801 non-null    float64       
 8   Loss on Ignition      801 non-null    float64       
 9   Blaine                801 non-null    float64       
 10  #200                  801 non-null    float64       
 11  #325                  801 non-null    float64       
 12  Initial setting time  801 non-null    float64       
 13  Final setting time  

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [39]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [40]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [41]:
df.to_csv("../../../../data/interim/204/aq.csv", index=False)

<IPython.core.display.Javascript object>